**Setting up Spark Enviorment**

**1, Deploy a Spark Cluster (Dataproc ,EMR,HDInsight ,on-prem)**

**2\. Store Data in HDFS and not on local**

**Download Source - "https://www.kaggle.com/****datasets/olistbr/brazilian-ecommerce****"**

**3\. use Pyspark to interat with data**

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
.appName('Module 1- Data_ingestion')\
.master('yarn')\
.getOrCreate()

25/08/22 05:27:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
spark

In [3]:
!hadoop fs -ls /tmp/Brazilian/

Found 9 items
-rw-r--r--   2 harshvardhansahu06733 hadoop    9033957 2025-08-21 20:04 /tmp/Brazilian/olist_customers_dataset.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop   61273883 2025-08-21 20:04 /tmp/Brazilian/olist_geolocation_dataset.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop   15438671 2025-08-21 20:04 /tmp/Brazilian/olist_order_items_dataset.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop    5777138 2025-08-21 20:04 /tmp/Brazilian/olist_order_payments_dataset.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop   14451670 2025-08-21 20:04 /tmp/Brazilian/olist_order_reviews_dataset.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop   17654914 2025-08-21 20:04 /tmp/Brazilian/olist_orders_dataset.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop    2379446 2025-08-21 20:04 /tmp/Brazilian/olist_products_dataset.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop     174703 2025-08-21 20:04 /tmp/Brazilian/olist_sellers_dataset.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop       2613 2025

In [4]:
hdfs_path= '/tmp/Brazilian'

In [14]:
customers_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_customers_dataset.csv')

geolocation_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_geolocation_dataset.csv')

order_items_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_order_items_dataset.csv')

order_payments_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_order_payments_dataset.csv')

order_reviews_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_order_reviews_dataset.csv')

orders_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_orders_dataset.csv')

products_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_products_dataset.csv')

sellers_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_sellers_dataset.csv')



In [19]:
customers_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [20]:
orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



# Data Leakage or Drop

In [17]:
# check it with kaggle page for the data set
print(f'customers : {customer_df.count()} rows')
print(f'orders : {orders_df.count()} rows')

customers : 99441 rows
orders : 99441 rows


# check for null Values

In [34]:
#Null or say duplicate values 

from pyspark.sql.functions import *

customers_df.select([count(when(col(c).isNull(),1)).alias(c) for c in customers_df.columns]).show()


+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|          0|                 0|                       0|            0|             0|
+-----------+------------------+------------------------+-------------+--------------+



# check for duplicate values

In [39]:
# duplicate value 
customers_df.groupBy('customer_id').count().filter('count>1').show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
+-----------+-----+



# customer Distribution by sales state

In [44]:
customers_df.groupBy('customer_state').count().orderBy(col('count').desc()).show()

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            SP|41746|
|            RJ|12852|
|            MG|11635|
|            RS| 5466|
|            PR| 5045|
|            SC| 3637|
|            BA| 3380|
|            DF| 2140|
|            ES| 2033|
|            GO| 2020|
|            PE| 1652|
|            CE| 1336|
|            PA|  975|
|            MT|  907|
|            MA|  747|
|            MS|  715|
|            PB|  536|
|            PI|  495|
|            RN|  485|
|            AL|  413|
+--------------+-----+
only showing top 20 rows



In [45]:
# order status distribution 

In [49]:
orders_df.groupBy('order_status').count().orderBy(col('count').desc()).show()

+------------+-----+
|order_status|count|
+------------+-----+
|   delivered|96478|
|     shipped| 1107|
|    canceled|  625|
| unavailable|  609|
|    invoiced|  314|
|  processing|  301|
|     created|    5|
|    approved|    2|
+------------+-----+



In [53]:
# Payments 
order_payments_df.show(5)




+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
+--------------------+------------------+------------+--------------------+-------------+
only showing top 5 rows



In [54]:
# top selling products 

In [66]:
order_items_df.show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [62]:
order_items_df.groupBy('product_id').count().orderBy(col('count').desc()).show(5)

+--------------------+-----+
|          product_id|count|
+--------------------+-----+
|aca2eb7d00ea1a7b8...|  527|
|99a4788cb24856965...|  488|
|422879e10f4668299...|  484|
|389d119b48cf3043d...|  392|
|368c6c730842d7801...|  388|
+--------------------+-----+
only showing top 5 rows



In [64]:
#total sales

In [69]:
order_items_df.groupBy('product_id').agg(sum('price').alias('total_sales')).orderBy(col('total_sales').desc()).show(5)

+--------------------+------------------+
|          product_id|       total_sales|
+--------------------+------------------+
|bb50f2e236e5eea01...|           63885.0|
|6cdd53843498f9289...| 54730.20000000005|
|d6160fb7873f18409...|48899.340000000004|
|d1c427060a0f73f6b...| 47214.51000000006|
|99a4788cb24856965...|43025.560000000085|
+--------------------+------------------+
only showing top 5 rows



# Average delivery time analysis

In [82]:
orders_df.show(10)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [84]:
dates_df = orders_df.select( 'order_id','order_purchase_timestamp','order_delivered_customer_date')

In [85]:
dates_df.show(5)

+--------------------+------------------------+-----------------------------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|
+--------------------+------------------------+-----------------------------+
|e481f51cbdc54678b...|     2017-10-02 10:56:33|          2017-10-10 21:25:13|
|53cdb2fc8bc7dce0b...|     2018-07-24 20:41:37|          2018-08-07 15:27:45|
|47770eb9100c2d0c4...|     2018-08-08 08:38:49|          2018-08-17 18:06:29|
|949d5b44dbf5de918...|     2017-11-18 19:28:06|          2017-12-02 00:28:42|
|ad21c59c0840e6cb8...|     2018-02-13 21:18:39|          2018-02-16 18:17:02|
+--------------------+------------------------+-----------------------------+
only showing top 5 rows



In [94]:
average_delivery_time= dates_df.withColumn('average time', datediff('order_delivered_customer_date','order_purchase_timestamp'))

In [95]:
average_delivery_time.agg(mean('average time')).show()

+------------------+
| avg(average time)|
+------------------+
|12.497336125046644|
+------------------+



In [97]:
average_delivery_time.orderBy(col('average time').desc()).show(5)

+--------------------+------------------------+-----------------------------+------------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|average time|
+--------------------+------------------------+-----------------------------+------------+
|ca07593549f1816d2...|     2017-02-21 23:31:27|          2017-09-19 14:36:39|         210|
|1b3190b2dfa9d789e...|     2018-02-23 14:57:35|          2018-09-19 23:24:07|         208|
|440d0d17af552815d...|     2017-03-07 23:59:51|          2017-09-19 15:12:50|         196|
|2fb597c2f772eca01...|     2017-03-08 18:09:02|          2017-09-19 14:33:17|         195|
|285ab9426d6982034...|     2017-03-08 22:47:40|          2017-09-19 14:00:04|         195|
+--------------------+------------------------+-----------------------------+------------+
only showing top 5 rows

